In [55]:
import math
import time
import pickle
import os
import glob
import numpy as np
from multiprocessing import Pool

device = 'cpu'
data_dir = 'data'

if device == 'cpu' :
    import numpy.random as random
    pool_n = 10
    
    randn = lambda x : random.randn(x)
    relu = lambda x : np.maximum(x,0)
    exp = lambda x : np.exp(x)
    mean = lambda x : np.mean(x)
    set_seed = lambda x : random.seed(x)
    
else :
    import torch
    pool_n = 1
    
    randn = lambda x : torch.randn(x)
    relu = lambda x : torch.relu(x)
    exp = lambda x : torch.exp(x)
    mean = lambda x : torch.mean(x)
    set_seed = lambda x : torch.manual_seed(x)

In [56]:
train_file_num = 300
test_file_num = 100
N = 100000

In [57]:
def MC(vol,K,T,N) :
    
    S0 = 1.
    W = randn(N)*np.sqrt(T)
    S = S0*exp( -0.5*vol**2*T + vol*W )
        
    p = relu(S-K)
    payoff = np.maximum(S0-K,0.)
    tv = mean(p)-payoff
    
    if not device == 'cpu' :
        tv = tv.cpu().detach()
    return tv

In [58]:
def make_data_impl(arg) :
    
    rank,data_num,N = arg 
    
    ns = time.time_ns()
    seed = int(time.time())+3*rank**2+5*rank
    set_seed(seed)
         
    vol = np.random.uniform(0.01,1.,data_num)
    T = np.random.uniform(0.01,1.,data_num)
    K0 = np.random.uniform(-2.,2.,data_num)
    K = K0*vol*np.sqrt(T)
    K = np.exp(K)

    tv = []
    for i,(vol_,K_,T_) in enumerate(zip(vol,K,T)) :
        tv.append( MC(vol_,K_,T_,N) )
    
    x = np.vstack([vol,K0,T]).T
    y = np.array(tv).reshape(-1,1)
    return (x,y)

def make_data(data_num,N) :
    with Pool(pool_n) as p:  
        results = p.map(make_data_impl,[(rank,data_num//pool_n,N) for rank in range(pool_n)]) 
    x = [e[0] for e in results]; x = np.array(x).reshape(-1,3)
    y = [e[1] for e in results]; y = np.array(y).reshape(-1,1)
    return (x,y)    

In [59]:
%%time
for mode,file_num in zip(['train','test'],[train_file_num,test_file_num]) :
    
    print(mode)
    if not os.path.exists(os.path.join(data_dir,mode)) :
        os.mkdir(os.path.join(data_dir,mode))
    filelist = glob.glob(f'{data_dir}/{mode}/bs_{mode}_*.pkl')
    start = len(filelist)

    for i in range(start,file_num) :
        print(i,end=' ')
        x,y = make_data(10000,N)
        with open(f'{data_dir}/{mode}/bs_{mode}_{N:.0e}_{i:05d}.pkl','wb') as f: 
            pickle.dump([x,y],f)
    print('')
    print('-'*50)

train
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 
--------------------------------------------------
test

--------------------------------------------------
CPU times: user 699 ms, sys: 3 s, total: 3.7 s
Wall time: 4min 7s
